In [1]:
import numpy as np
import pandas as pd
import time
import os

1. Open the overall.csv and extract x, y, z and store them into separate class
2. Sum the occupency together by id

In [2]:
def merge_csv(path, total_rank=128):
    start = time.time()
    column = ['x', 'y', 'z', 'occ']
    df_list = []
    for i in range(total_rank):
        step = f"{i:07}"
        df_temp = pd.read_csv('{}/pedata{}.csv'.format(path, step), header=None)
        df_temp = df_temp.drop(labels=[0, 4, 5, 6, 7, 8, 9, 10], axis=1)
        df_temp.columns = column
        df_list.append(df_temp)
    df_overall = pd.concat(df_list)
    df_overall.to_csv('{}/overall_py.csv'.format(path), index=False)
    end = time.time()
    print('merge data successfully in {}'.format(path))
    print('used {:.5f}s'.format(end - start))
    return 0

In [3]:
path_list = [ f for f in os.listdir() if f[0]=='l']
df = merge_csv(path_list[0])

merge data successfully in lbm_a2p95_s40_n230p0_n2p47_2p47.iter04060000
used 19.87817s


In [4]:
start = time.time()
df_t = pd.read_csv(path_list[0] + '/overall_py.csv')
end = time.time()
print('used {:.5f}s'.format(end - start))

used 1.16497s


In [5]:
lx = df_t.x.drop_duplicates()
ly = df_t.y.drop_duplicates()
lz = df_t.z.drop_duplicates()

In [6]:
occ_not_zero = df_t[df_t['occ'] != 0]
new_occ = [['coord', 'occ']]
for index, row in occ_not_zero.iterrows():
    new_occ.append([(row['x'], row['y'], row['z']), row['occ']])
new_occ = pd.DataFrame(new_occ)
new_header = new_occ.iloc[0] #grab the first row for the header
new_occ = new_occ[1:] #take the data less the header row
new_occ.columns = new_header 
new_occ = new_occ.reset_index(drop=True)
new_occ

,coord,occ
0,"(260.50410000000005, -4.462399999999988, -0.71...",0.0
1,"(260.9541, -4.462399999999988, -0.716140000000...",0.000001
2,"(261.4041, -4.462399999999988, -0.716140000000...",0.0
3,"(261.8541000000001, -4.462399999999988, -0.716...",0.0
4,"(262.3041, -4.462399999999988, -0.716140000000...",0.0
...,...,...
3101,"(263.2041, 0.9376000000000176, -0.716140000000...",0.000006
3102,"(262.75410000000005, 0.9376000000000176, -0.71...",0.000013
3103,"(262.3041, 0.9376000000000176, -0.716140000000...",0.000003
3104,"(261.8541000000001, 0.9376000000000176, -0.716...",-0.000001


In [ ]:
cube = {}
xx = 0
for x in lx:
    for y in ly:
        for z in lz:
            cube[str((x, y, z))] = 0.0
            xx += 1
print(xx)

In [ ]:
len(cube)

In [ ]:
for i in range(len(new_occ)):
    coord = new_occ['coord'][i]
    cube[str(coord)] = new_occ['occ'][i]
    if (((10*i)/len(new_occ)) % 10) == 0:
        print(((10*i)/len(new_occ)) % 10)
        print('10% done')

In [ ]:
# cube = []
# cube.append(['coord', 'occ'])
# for x in lx:
#     for y in ly:
#         for z in lz:
#             cube.append([(x, y, z), 0])
# df_cube = pd.DataFrame(cube)
# new_header = df_cube.iloc[0] #grab the first row for the header
# df_cube = df_cube[1:] #take the data less the header row
# df_cube.columns = new_header 
# df_cube = df_cube.reset_index(drop=True)

In [ ]:
# write csv to vtk
start = time.time()

file = open('0_n3p50_OPF.vtk', 'w')
file.write('# vtk DataFile Version 3.0\n')
file.write('vtk global output\n')
file.write('ASCII\n')
file.write('DATASET STRUCTURED_POINTS\n')
file.write("DIMENSIONS {} {} {}\n".format(len(lx), len(ly), len(lz)))
file.write("ORIGIN 0. 0. 0.\n");
file.write("SPACING 1 1 1\n");
file.write("POINT_DATA {}\n".format(len(cube)));
file.write('SCALARS occ double \n')
file.write("LOOKUP_TABLE default\n")
for entry in cube:
    occ = cube[entry]
    line = '{}\n'.format(occ)
    file.write(line)
file.write('VECTORS coordinates double \n')
ss = 0
for entry in cube:
    entry_list = entry.split(", ")
    x = entry_list[0][1:]
    y = entry_list[1]
    z = entry_list[2][:-1]
    line = '{} {} {}\n'.format(x, y, z)
    ss += 1
    file.write(line)
file.close
print(ss)

end = time.time()
print('used {:.5f}s'.format(end - start))